# ROAD Development Notebook

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
import os
print(os.cpu_count())

8


In [ ]:
!pip install numpy==1.23.1
!pip install numba==0.56.4
!pip install scikit-learn==1.2.2
!pip install XGBoost==1.5.0
!pip install shap==0.39.0
!pip install miceforest
!pip install multiprocess
!pip install joblib==1.2.0
!pip install shapflow

### Libraries

In [16]:
# Author: Eddie Perez Claudio
# The goal of this script is to evaluate the performance of XAI methods on the simulated datasets using the ROAD method.

## Import Libraries ##

#XAI
import shap
import shapflow.flow as flow

#Imputation
import miceforest as mf

# Data wrangling
import itertools
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split



## Libraries ##
#General
import numpy as np
from numpy.random import Generator, PCG64
# import pandas as pd
import copy



#Model Evalutation
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, roc_auc_score, average_precision_score, recall_score

# multiprocessing
from multiprocess.pool import Pool
import os

## SHAP Benchmark (Remove and Debias)

In [17]:
#### Remove and Retrain ####
# This is a version of ROAD (as described in Rong et al 2022)
# which works with any scikit-learn model and tabular data (pandas dataframes)
#### Code ####

## Explanation Function ##
# Trains a model, builds an explanation with KernelShap, and then
# outputs the ranking of each feature in descending order.
# Arguments:
# clf : A trained ML model with a .predict method
# X: Training data as pandas dataframe
# x:  Test data as pandas dataframe
# explainer: any explainer that follows the shap api format
def explain(clf, X, x, explainer = shap.explainers.Permutation):
    X = X.to_numpy()
    x = x.to_numpy()

    if explainer == shap.KernelExplainer:
        try:
                    # #Build Explanation
            explanation = explainer(clf.predict_proba, shap.kmeans(X, 4))
            shap_values = np.abs(explanation.shap_values(x)[1]).mean(0)
            return shap_values

        except:
        # #Build Explanation
            explanation = explainer(clf.predict, shap.kmeans(X, 4))
            shap_values = np.abs(explanation.shap_values(x)).mean(0)
            return shap_values

    elif explainer == shap.explainers.Tree:
        # #Build Explanation
        explanation = explainer(clf, X)
        shap_values = explanation(x).values
        if len(shap_values.shape) == 2:
            shap_values = np.abs(shap_values).mean(0)
        else:
            shap_values = np.abs(shap_values[...,1]).mean(0)
        return shap_values

    elif explainer == shap.explainers.Linear:
        # #Build Explanation
        explanation = explainer(clf, X)
        shap_values = np.abs(explanation(x).values).mean(0)
        return shap_values

    elif explainer == flow.GraphExplainer:
        #If X is not pd.Dataframe, turn into one
        if type(X) != pd.DataFrame:
          if X.shape[1] == 6:
              bg = pd.DataFrame(X.mean(axis = 0).reshape(1,-1), columns=['HC1', 'HC2', 'C1', 'C2', 'C3', 'Noise'])
              X_df = pd.DataFrame(X, columns=['HC1', 'HC2', 'C1', 'C2', 'C3', 'Noise'])
              x_df = pd.DataFrame(x, columns=['HC1', 'HC2', 'C1', 'C2', 'C3', 'Noise'])
          elif X.shape[1] == 5:
              bg = pd.DataFrame(X.mean(axis = 0).reshape(1,-1), columns=['HC1', 'C1', 'C2', 'C3', 'Noise'])
              X_df = pd.DataFrame(X, columns=['HC1', 'C1', 'C2', 'C3', 'Noise'])
              x_df = pd.DataFrame(x, columns=['HC1', 'C1', 'C2', 'C3', 'Noise'])
          elif X.shape[1] == 4:
              bg = pd.DataFrame(X.mean(axis = 0).reshape(1,-1), columns=['C1', 'C2', 'C3', 'Noise'])
              X_df = pd.DataFrame(X, columns=['C1', 'C2', 'C3', 'Noise'])
              x_df = pd.DataFrame(x, columns=['C1', 'C2', 'C3', 'Noise'])
        #Build Graph Structure
        causal_links = flow.CausalLinks()
        categorical_feature_names = list([])
        display_translator = flow.translator(X_df.columns, X_df, X_df)
        target_name = 'Predicted Target'
        feature_names = list(['C1', 'C2', 'C3', 'Noise'])
        causal_links.add_causes_effects(feature_names, target_name, create_f(feature_names, clf.predict))

        if X.shape[1] == 6:
          hidden_causes = ['HC1', 'HC2']
          effects = ['C1', 'C2', 'C3']
          causal_links.add_causes_effects(hidden_causes, effects)
        elif X.shape[1] == 5:
          hidden_causes = ['HC1']
          effects = ['C1', 'C2', 'C3']
          causal_links.add_causes_effects(hidden_causes, effects)

        graph = flow.build_feature_graph(X_df, causal_links, categorical_feature_names, display_translator, target_name)

        #Explain the graph

        graph_exp = flow.GraphExplainer(graph, bg, nruns = 1000)
        explanation = graph_exp.shap_values(x_df)
        shap_values = pd.DataFrame(flow.node_dict2str_dict(explanation.edge_credit)).applymap(lambda x: np.mean(np.abs(x)))
        shap_values = shap_values.sort_index(axis = 1)[['C1', 'C2', 'C3', 'Noise']].iloc[0].to_numpy()
        return shap_values





    else:
        try:
                    # #Build Explanation
            explanation = explainer(clf.predict_proba, X)
            shap_values = np.abs(explanation(x).values[...,1]).mean(0)
            return shap_values

        except:
        # #Build Explanation
            explanation = explainer(clf.predict, X)
            shap_values = np.abs(explanation(x).values).mean(0)
            return shap_values


## Ranking Function ##
# Outputs the ranking of each feature in descending order.
# Arguments:
# shap_values: the output of any shap explainer
def ranker(shap_values):
    values = copy.deepcopy(shap_values)
    #Get ranks
    ranks = np.argsort(values)
    return ranks


## Metrics function ##
# Utility function which outputs a series of metrics to evaluate
# Currently gets accuracy, balanced_accuracy, and f-score

#Arguments:
#clf: A trained ML model with a predict method
#x: a pd.DataFrame of test data
#y: Target of the test data
def metrics(clf, x, y):
    #Predict
    # print(len(x))
    yhat = clf.predict(x.to_numpy())
    try:
        yscore = clf.predict_proba(x.to_numpy())[:, 1]
    except:
        yscore = clf.decision_function(x.to_numpy())
    #Get metrics
    accu = accuracy_score(y, yhat)
    accu_balanced = balanced_accuracy_score(y, yhat)
    f1 = f1_score(y, yhat)
    auroc = roc_auc_score(y, yscore)
    auprc = average_precision_score(y, yscore)
    recall = recall_score(y, yhat)

    return np.array([[accu], [accu_balanced], [f1], [auroc], [auprc], [recall]])



## Imputation Function assistant ##

def impute(i,k, rankings, x_test):
       #Add NA values
        x_test = mf.ampute_data(x_test, variables= list(x_test.columns[rankings[i:k]]), perc = 0.8, random_state=42)
        #impute
        variables = x_test.columns.to_list()
        index = rankings.copy()[i:k].tolist().pop()
        kds = mf.ImputationKernel(
        x_test,
        variable_schema={variables.pop(index) : variables},
        save_all_iterations=True,
        random_state=1991
        )
        # Run the MICE algorithm for 2 iterations
        kds.mice(2)
        # Return the completed dataset.
        x_test = kds.complete_data()

        return x_test


## Mask Features ##
#These functions mask data using imputation and retrain the ML model
#They mask from the top %, bottom %, or random.

# Arguments:
# t: percentage or number of features to be masked in each iteration
# rankings: a list of rankings (descending rankings) to guide the removal
# X, x: dataframes of the data from which we will remove the features. Training set and testing set, respectively
# Y, y: Targets for the training and testing sets, respectively
#clf: Model to retrain
# base: metrics of the full model. Used to compare the performance of the masked model
# direction: direction of masking. Can be 'top', 'bottom', or 'random'
# seed: seed for random number generator

#Mask
def mask(t, rankings, X, Y, x, y, clf, base = np.empty((3,1)), direction = 'top'):
    #Make copies of our data to modify
    X_train = copy.deepcopy(X)
    x_test = copy.deepcopy(x)
    results = copy.deepcopy(base)

    ## Directional Masking ##
    #Top
    if direction == 'top':
        #Set masking schedule and iterator
        if type(t) != int:
            j = int(np.round(len(rankings)*t))
            i = len(rankings) - j
            k = len(rankings)
        else:
            j = t
            i = len(rankings) - t
            k = len(rankings)

        #Impute and Predict
        while k >= j:
            #Impute
            x_test = impute(i,k, rankings, x_test)
            #Predict
            results =  np.hstack((results, metrics(clf, x_test, y)))

            #Move iterator forward
            i -= j
            k -= j

        return results

    #Bottom
    elif direction == 'bottom':
        #Set masking schedule
        if type(t) != int:
            j = int(np.round(len(rankings)*t))
            i = 0
            k = j
        else:
            j = t
            i = 0
            k = j

        #Impute and Predict
        while k <= len(rankings):
            #Impute
            x_test = impute(i,k, rankings, x_test)
            #Predict
            results =  np.hstack((results, metrics(clf, x_test, y)))

            #Move iterator forward
            i += j
            k += j
        return results

    #Random
    elif direction == 'random':
        rng = Generator(PCG64())
        random_choices = rng.permutation(rankings)
        #Set masking schedule
        if type(t) != int:
            j = int(np.round(len(rankings)*t))
            i = 0
            k = j
        else:
            j = t
            i = 0
            k = j

        #Impute and Predict
        while k <= len(rankings):
            #Impute
            x_test = impute(i,k, random_choices, x_test)
            #Predict
            results =  np.hstack((results, metrics(clf, x_test, y)))

            #Move iterator forward
            i += j
            k += j
        return results






## ROAD ##
# The main function of the library.
#Wraps all other functions in a nice pipeline which is easy to use.
#Accepts any scikit-learn model. It was built and tested using a binary target.

# Arguments:
# model : the model to be re-trained
# t: percentage of features to be removed in each iteration
# X: Training data as pandas dataframe
# Y: Target values for training (This was build using a binary target)
# x:  Test data as pandas dataframe
# y: Target values for testing
# explainer: any explainer which built with the shap api
# repeats: how many times to explain and do the whole retraining

#outputs accuracy, balanced_accuracy, f1_score, and ranks for each iteration.
def road(X, Y, x, y, model, explainer = None, t = 0.10, shap_values = None):
    #Initialize variables
    base = metrics(model, x[['C1', 'C2', 'C3', 'Noise']], y)
    if explainer != None:
        values = explain(model, X, x, explainer)
        ranks = ranker(values)

    elif shap_values != None:
        values = shap_values
        ranks = ranker(values)
    else:
        print('Must supply either an explainer or shap_values')
        return

    top = mask(t, ranks, X, Y, x[['C1', 'C2', 'C3', 'Noise']], y, model, base, direction='top')

    bottom = mask(t, ranks, X, Y, x[['C1', 'C2', 'C3', 'Noise']], y, model, base, direction='bottom')

    random = mask(t, ranks, X, Y, x[['C1', 'C2', 'C3', 'Noise']], y, model, base, direction='random')



    # #Set progress bar
    # # Repeat x times
    # for i in range(repeats-1):
    #             #Initialize
    #     if explainer != None:
    #         iter_values = explain(model, X, x, explainer)
    #         ranks = ranker(iter_values)
    #         values = np.dstack((values, iter_values))
    #     elif shap_values != None:
    #         ranks = ranker(values)
    #     else:
    #         print('Must supply either an explainer or shap_values')
    #         return
    #     top = np.dstack((top, mask(t, ranks, X, Y, x, y, model, base, direction='top')))
    #     bottom = np.dstack((bottom, mask(t, ranks, X, Y, x, y, model, base, direction='bottom')))
    #     random = np.dstack((random, mask(t, ranks, X, Y, x, y, model, base, direction='random')))
    return [top, bottom, random, values]


# Model Wrapper for ShapFlow compatibility
def create_f(parents, m, **kwargs):
    '''
    assume m is a linear regression model from sklearn predict / predict_proba
    parents are list of feature names
    kwargs: keyword args that applies to the predict function
    '''
    def f_(*args):
        return m(np.vstack(args).T)

    return f_


## Faux Model

In [18]:
## Build a faux model for the data
## This will only work for ShapSampling and ShapKernel, or ShapPermutation

class faux_model:
  def __init__(self):
      self.name = 'FauxModel'

  @staticmethod
  def predict(x):
    if str(type(x)).__contains__('pandas'):
      x = x.to_numpy()

    return (x[:,0:3].sum(axis = 1)/3 > 0.5).astype(int)

  def predict_proba(self, x):
    yhat = self.predict(x)
    return np.array([np.logical_not(yhat),yhat]).T.astype(int)


# y = data['3_vars_corr_2HC_n10000.csv']['y_test']
# x = data['3_vars_corr_2HC_n10000.csv']['X_test']

## Model Eval

### Imports

In [19]:
### Import Data ###
# Simulated Datasets

# Path to Simulated Datasets

dir_path = '/content/drive/MyDrive/Data-ML-XAI-Eval/Synthetic Data/Data files with summed target'

# Lists to store files name
res_ = []
res = []
for (dir_path, dir_names, file_names) in os.walk(dir_path):
    res_.extend(file_names)
for file in res_:
    if file not in ['GroundTruth.csv', '.Rhistory', '.DS_Store', '3_vars_corr_2HC_n30000.csv']:
        res.append(file)

print(res)

data = {}
for file in res:
    data_path = f'{dir_path}/{file}'
    df = pd.read_csv(data_path)
    train, test = train_test_split(df, test_size=0.25, random_state=42)
    data[file] = {
        'X_train': train.drop('Target', axis=1),
        'y_train': train.Target,
        'X_test': test.drop('Target', axis=1),
        'y_test': test.Target
    }


### Import Models ###
# Import Models
names = ['RandomForest', 'LogisticRegression', 'XGBoost', 'FauxModel']
file_models = {}
for file in res:
    models = {}
    for model in names:
        if model == 'FauxModel':
            models[model] = faux_model()
        else:
            models[model] = joblib.load(
                f'/content/drive/MyDrive/Data-ML-XAI-Eval/Model Training/Models Sum/{file}_{model}.joblib')
    file_models[file] = models

### Model types ###
# Linear Models
linear = ['PassiveAgressive', 'SGDClassifier', 'Perceptron', 'RidgeClassifier', 'LogisticRegression']
# Tree Models
tree = ['RandomForest', 'DecisionTree', 'XGBoost']
# Other
other = ['MultinomialNB', 'GaussianNB', 'FauxModel']

### Set Explainers ###
# Explainers of Interest
xai = {
    'ShapLinear': shap.explainers.Linear,
    'ShapTree': shap.explainers.Tree,
    'ShapPermutation': shap.explainers.Permutation,
    'ShapSampling': shap.explainers.Sampling,
    'ShapExact' : shap.explainers.Exact,
    'ShapKernelKmeans': shap.explainers._kernel.Kernel,
    'ShapFlow': flow.GraphExplainer
}

### Define Parallel Function ###
def parallel_road(file, model_name, exp, repeat):

    #Check if path exists
    path = f"/content/drive/MyDrive/Data-ML-XAI-Eval/Model Explanation/ROAD Output/{file}_{model_name}_{exp}_{repeat}.joblib"

    try:
      top, bottom, random, values = joblib.load(path)
    except:
      values = None

    if not (values is None):
      return

    else:
      # print(path)
      if os.path.isfile(path):
        os.remove(path)

      df = data[file]
      models = file_models[file]
      model = models[model_name]
      save_path = '/content/drive/MyDrive/Data-ML-XAI-Eval/Model Explanation/ROAD Output'
      # res_ = {}
      # evaluate linear exps
      if (model_name in linear) and (exp in ['ShapSampling', 'ShapLinear', 'ShapPermutation']):
              res_ = road(df['X_train'], df['y_train'], df['X_test'],
                                    df['y_test'], model=model, t=1,  explainer=xai[exp])
              joblib.dump(res_, f'{save_path}/{file}_{model_name}_{exp}_{repeat}.joblib', compress=3)

      # eval tree exps
      elif (model_name in tree) and (exp in ['ShapSampling', 'ShapTree', 'ShapPermutation']):
              res_ = road(df['X_train'], df['y_train'], df['X_test'],
                                    df['y_test'], model=model, t=1,  explainer=xai[exp])
              joblib.dump(res_, f'{save_path}/{file}_{model_name}_{exp}_{repeat}.joblib', compress=3)

      # ev al agnostic exps
      elif (exp in ['ShapSampling', 'ShapPermutation', 'ShapExact', 'ShapKernelKmeans', 'ShapFlow']):
              res_ = road(df['X_train'], df['y_train'], df['X_test'],
                                    df['y_test'], model=model, t=1,  explainer=xai[exp])
              joblib.dump(res_, f'{save_path}/{file}_{model_name}_{exp}_{repeat}.joblib', compress=3)
      else:
        return

      print(f'\n {file}_{model_name}_{exp}_{repeat} done \n')
      #joblib.dump(res_, f'/content/drive/MyDrive/XAI method performacne when Explainaing the PORT Dataset/Results/Evaluations/ROAD/SimOptim/{file}_{model_name}_optimized.joblib', compress=3)
      return

print('built')

['2_vars_corr_2HC_n700.csv', '0_vars_corr_0HC_n700.csv', '3_vars_corr_1HC_n1000.csv', '2_vars_corr_1HC_n900.csv', '3_vars_corr_2HC_n700B.csv', '2_vars_corr_2HC_n500.csv', '0_vars_corr_0HC_n500.csv', '3_vars_corr_2HC_n900B.csv', '0_vars_corr_0HC_n100.csv', '2_vars_corr_2HC_n100.csv', '3_vars_corr_2HC_n500B.csv', '0_vars_corr_0HC_n300.csv', '2_vars_corr_2HC_n300.csv', '1_vars_corr_1HC_n900.csv', '3_vars_corr_2HC_n300.csv', '3_vars_corr_2HC_n100B.csv', '0_vars_corr_0HC_n400.csv', '0_vars_corr_0HC_n600.csv', '3_vars_corr_2HC_n100.csv', '3_vars_corr_2HC_n500.csv', '0_vars_corr_0HC_n200.csv', '3_vars_corr_2HC_n1000.csv', '3_vars_corr_1HC_n900.csv', '3_vars_corr_2HC_n700.csv', '3_vars_corr_2HC_n300B.csv', '3_vars_corr_1HC_n500.csv', '3_vars_corr_1HC_n700.csv', '3_vars_corr_2HC_n900.csv', '3_vars_corr_1HC_n300.csv', '0_vars_corr_0HC_n800.csv', '3_vars_corr_1HC_n100.csv', '2_vars_corr_1HC_n1000.csv', '0_vars_corr_0HC_n1000.csv', '1_vars_corr_1HC_n500.csv', '1_vars_corr_1HC_n1000.csv', '2_vars_c

### Set-Up

In [20]:
ml = ['RandomForest', 'LogisticRegression', 'XGBoost', 'FauxModel']
ml = ['RandomForest']
exp = list(xai.keys())
print(exp)
repeat = list(range(20))[10:]
print(repeat)

# #Select files with skew:
# files = []
# for file in res:
#  if ('1_' in file) and '700' in file and '1000' not in file:
#   files.append(file)
# files

['ShapLinear', 'ShapTree', 'ShapPermutation', 'ShapSampling', 'ShapExact', 'ShapKernelKmeans', 'ShapFlow']
[10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


### Run


In [ ]:
## Paralelization ##
if __name__ == '__main__':
  pool = Pool(8)
  results = pool.starmap_async(parallel_road, itertools.product(res, ml, exp, repeat), chunksize=1)
  results.get()


 2_vars_corr_2HC_n700.csv_RandomForest_ShapTree_15 done 


 2_vars_corr_2HC_n700.csv_RandomForest_ShapTree_11 done 


 2_vars_corr_2HC_n700.csv_RandomForest_ShapTree_13 done 


 2_vars_corr_2HC_n700.csv_RandomForest_ShapTree_12 done 


 2_vars_corr_2HC_n700.csv_RandomForest_ShapTree_14 done 


 2_vars_corr_2HC_n700.csv_RandomForest_ShapTree_17 done 


 2_vars_corr_2HC_n700.csv_RandomForest_ShapTree_16 done 


 2_vars_corr_2HC_n700.csv_RandomForest_ShapTree_18 done 


 2_vars_corr_2HC_n700.csv_RandomForest_ShapTree_10 done 



Permutation explainer:   2%|▏         | 4/182 [00:11<00:37,  4.76it/s]


 2_vars_corr_2HC_n700.csv_RandomForest_ShapTree_19 done 



Permutation explainer:  97%|█████████▋| 177/182 [00:50<00:01,  4.45it/s]
Permutation explainer: 183it [00:53,  3.41it/s]
Permutation explainer: 183it [00:52,  3.42it/s]
Permutation explainer: 183it [00:53,  3.41it/s]
Permutation explainer: 183it [00:53,  3.33it/s]
Permutation explainer: 183it [00:51,  3.27it/s]
Permutation explainer: 183it [00:53,  3.26it/s]
Permutation explainer: 183it [00:55,  3.05it/s]



 2_vars_corr_2HC_n700.csv_RandomForest_ShapPermutation_11 done 


 2_vars_corr_2HC_n700.csv_RandomForest_ShapPermutation_10 done 



...

Permutation explainer:  20%|█▉        | 49/251 [00:13<00:52,  3.85it/s]


 2_vars_corr_1HC_n900.csv_RandomForest_ShapTree_19 done 



bruteforce sampling:  36%|███▋      | 365/1000 [01:30<01:42,  6.17it/s]


 2_vars_corr_1HC_n900.csv_RandomForest_ShapTree_18 done 



Permutation explainer:  32%|███▏      | 80/251 [00:17<00:38,  4.40it/s]